<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/DGRNet%20-%20Last%20Step%20_Comparison%20Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
import keras.backend as K
from keras.callbacks import Callback
import plotly
import plotly.express as px # for data visualization
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import numpy as np
import scipy.stats as stats
from numpy import array
from numpy import hstack
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV, MultiTaskLassoCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import TimeSeriesSplit
from math import sqrt
from sklearn.linear_model import LinearRegression
from keras.models import Model
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

#-----------------------------------------------------GET DATA - X, Predicted Wndow from MLP, and Derived Labels, for the entire dataset that was gennerated through VAE-----------------------------------

#y_pred = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/predicted_COSW.csv')
#y_true = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/Calculated_label_COSW.csv')
#x_orig = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/whole_data.csv')

y_pred = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/predicted_window_V3.csv')
y_true = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/derived_window_label_V3.csv')
x_orig = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/test_data_V3.csv')

n_features = 6
window_size = int(x_orig.shape[1]/n_features)
x = x_orig.reshape((x_orig.shape[0],window_size,n_features))


#-----------------------------------------------CASES - pot window values and pick a few fixed window sizes -------------------------------------------

plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.title("Distribution before Transformation", fontsize=15)
sns.histplot(y_true, kde=True, color="red")
plt.subplot(1,2,2)


fx_window1 = 5
fx_window2 = 10
fx_window3 = 15
fx_window4 = 20
fx_window5 = 25

pred_step = 1

aic1_count = 0
aic2_count = 0
aic3_count = 0
aic4_count = 0
aic5_count = 0
aic6_count = 0




for i in range(x.shape[0]-1) :

  x1 = x[i]
  x_train=x1[:-pred_step]
  x_test=x1[-pred_step:]
  regressor= VAR(x_train)
  results1 = regressor.fit(fx_window1)
  results2 = regressor.fit(fx_window2)
  results3 = regressor.fit(fx_window3)
  results4 = regressor.fit(fx_window4)
  results5 = regressor.fit(fx_window5)
  results6 = regressor.fit(int(y_pred[i]))

#Calculate AIC for ach case--------------
  try:
    aic1 = results1.aic
  except:
    aic1 = 9999

  try:
    aic2 = results2.aic
  except:
    aic2 = 9999

  try:
    aic3 = results3.aic
  except:
    aic3 = 9999

  try:
    aic4 = results4.aic
  except:
    aic4 = 9999

  try:
    aic5 = results5.aic
  except:
    aic5 = 9999

  try:
    aic6 = results6.aic
  except:
    aic6 = 9999


#Now capture which case has least AIC------

  l = (aic1,aic2,aic3, aic4,aic5,aic6)
  # get index of smallest item in list
  winner = l.index(min(l))
  if winner == 0 :
    aic1_count += 1

  elif winner == 1 :
    aic2_count += 1
  elif winner == 2 :
    aic3_count += 1
  elif winner == 3 :
    aic4_count += 1
  elif winner == 4 :
    aic5_count += 1
  elif winner == 5 :
    aic6_count += 1
  else :
    print("error")



print("aic1_count : ", aic1_count)
print("aic2_count : ", aic2_count)
print("aic3_count : ", aic3_count)
print("aic4_count : ", aic4_count)
print("aic5_count : ", aic5_count)
print("aic6_count : ", aic6_count)

l1 = (aic1_count,aic2_count,aic3_count, aic4_count, aic5_count, aic6_count)
Winner_ind = l1.index(max(l1))
print("The Winner Is: ", l1[Winner_ind])




In [ ]:
from google.colab import drive
drive.mount('/content/drive')